# Coupon campaign optimization using PuLP

Author: William Javier Garcia Herrera - wilomaku@gmail.com

Linear optimization is a method to optimize problems (maximize or minimize a function) given some constraints. The goal is to find the optimal coupon that maximizes the expected return of the user.

A poorly designed coupon strategy can do serious harm to a business's profit margin [1]. The optimal coupon has a value as low as possible, guaranteeing conversion, and looking to the expected return of the user. The company should be more willing to give higher coupons to more valuable users.

Since I am more interested in the optimization part, the user's value and the probability of conversion given a certain coupon, will be generated randomly. In the practice, these values should be given for a model. I will use an example of 10 synthetic users who will receive a coupon (ranging from 4 to 12 <your currency\>). For optimization, I will use PuLP, a free open source software for Python to describe optimization problems as mathematical models [2,3].

In [1]:
# Loading libraries
import numpy as np
import datetime
import pulp as plp
import pandas as pd
from random import random, sample, choices

## Synthetic dataset

The dataset is created with probabilities for n customers and c coupons (coupon_c > coupon 1), and Customer Life Time Value (CLTV) per user. In a real scenario, the probabilities and the cltv can be estimated by a model but since we are interested in the optimization part, they will be simulated.

In [2]:
def create_customers(n, c):
    '''
        Generate synthetic n users with random probabilities for an array of coupons (c), and CLTV
    '''
    
    # Multiplier to generate random probability (multiplier assures that probability keeps increasing
    #   with higher coupons)
    mults = np.arange(1, n, 1)/100
    randommults = choices(mults, k=n)
    
    # Random CLTV between [30,120)
    cltvs = np.arange(30, 120, 1)
    randomcltvs = choices(cltvs, k=n)
    
    # DF appending probabilities and CLTV
    cols_prob_coupon = ['prob_coupon_{}'.format(i) for i in c]
    customers_df = pd.DataFrame(columns = cols_prob_coupon+['cltv'])
    
    # Loop to generate n synthetic users
    for i in range(n):
        probs = [j*randommults[i] for j in np.arange(1,len(c)+1,1)]
        probs.append(randomcltvs[i])
        customers_df.loc[i, cols_prob_coupon+['cltv']] = np.array(probs)
    
    return customers_df

# Uncomment to generate synthetic dataset
#coupons = np.arange(4,12,1)
#df = create_customers(10, coupons)
#df.to_csv('../data/synthetic_users.csv.gz', sep=';', index=False)

In [3]:
df = pd.read_csv('../data/synthetic_users.csv.gz', sep=';')
df

,prob_coupon_4,prob_coupon_5,prob_coupon_6,prob_coupon_7,prob_coupon_8,prob_coupon_9,prob_coupon_10,prob_coupon_11,cltv
0,0.06,0.12,0.18,0.24,0.30,0.36,0.42,0.48,98.0
1,0.06,0.12,0.18,0.24,0.30,0.36,0.42,0.48,82.0
2,0.03,0.06,0.09,0.12,0.15,0.18,0.21,0.24,112.0
3,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,91.0
4,0.06,0.12,0.18,0.24,0.30,0.36,0.42,0.48,80.0
5,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,87.0
6,0.04,0.08,0.12,0.16,0.20,0.24,0.28,0.32,35.0
7,0.06,0.12,0.18,0.24,0.30,0.36,0.42,0.48,40.0
8,0.04,0.08,0.12,0.16,0.20,0.24,0.28,0.32,46.0
9,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,80.0


## BAU approach

One possible estrategy to assign coupons without optimization is to give a coupon with a value equal to 10% of the user's cltv. This is the Business As Usual (BAU) approach. The Return Over Investment (ROI) of the campaign is calculated as the sum of the expected return of the users minus the total coupons invested value.

In [4]:
# Extracting pre-stablished coupon values
coupons = [int(col.split('_')[-1]) for col in df.columns if 'cltv' not in col]

# Assingned coupon = 10%*cltv 
coupons_bau = np.array(round(df['cltv']*0.1)).astype(int)

# Adding BAU columns
df['coupon_bau'] = coupons_bau
df['prob_coupon_bau'] = df.apply(lambda x: x['prob_coupon_{}'.format(int(x['coupon_bau']))], axis=1)
df['roi_bau'] = df['prob_coupon_bau']*(df['cltv'] - df['coupon_bau'])
df[['cltv','coupon_bau','prob_coupon_bau','roi_bau']]

,cltv,coupon_bau,prob_coupon_bau,roi_bau
0,98.0,10,0.42,36.96
1,82.0,8,0.30,22.20
2,112.0,11,0.24,24.24
3,91.0,9,0.06,4.92
4,80.0,8,0.30,21.60
5,87.0,9,0.06,4.68
6,35.0,4,0.04,1.24
7,40.0,4,0.06,2.16
8,46.0,5,0.08,3.28
9,80.0,8,0.05,3.60


In [5]:
# Revenue expected for BAU of coupon based on 10% of the CLTV
ROI_BAU = df['roi_bau'].sum()
print('Expected ROI for BAU: {}'.format(ROI_BAU))

Expected ROI for BAU: 124.88


## Optimization approach

The goal of the optimizer is to give a recommended coupon per user. In order to do that, we need: 1)Input matrices, 2) Constraints, 3) Objective function.

In [6]:
# Preparing potential coupons per user
set_users = list(df.index)
print('users: {}'.format(set_users))
print('coupons: {}'.format(coupons))

coupons_per_user = {i: coupons for i in set_users}

print('\tPossible coupons per user:\n')
coupons_per_user

users: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
coupons: [4, 5, 6, 7, 8, 9, 10, 11]
	Possible coupons per user:



{0: [4, 5, 6, 7, 8, 9, 10, 11],
 1: [4, 5, 6, 7, 8, 9, 10, 11],
 2: [4, 5, 6, 7, 8, 9, 10, 11],
 3: [4, 5, 6, 7, 8, 9, 10, 11],
 4: [4, 5, 6, 7, 8, 9, 10, 11],
 5: [4, 5, 6, 7, 8, 9, 10, 11],
 6: [4, 5, 6, 7, 8, 9, 10, 11],
 7: [4, 5, 6, 7, 8, 9, 10, 11],
 8: [4, 5, 6, 7, 8, 9, 10, 11],
 9: [4, 5, 6, 7, 8, 9, 10, 11]}

### Input matrices
Arrays of features used to describe the search space.

* x: Decision matrix stating the selected coupon that will be defined by the optimizer (x: [1,0])
* a: Coupon matrix
* c: Probability matrix
* d: Expected return (CLTV - coupon)
* b: Total budget constraint. It will be used the same budget as for the BAU (10% of the CLTV)

In [7]:
# Input matrices
print('Create matrices...\n')

x = dict()
a = dict()
c = dict()
d = dict()
b = (df['cltv'].sum())/10

for k, i in enumerate(set_users):
    for j in coupons_per_user[i]:
       
        x[(i, j)] = plp.LpVariable(cat=plp.LpInteger, lowBound=0, upBound=1, name="x_{0}_{1}".format(i, j))
        a[(i, j)] = j
        c[(i, j)] = df.loc[i]['prob_coupon_{}'.format(j)]
        d[(i,j)] = df.loc[i]['cltv'] - j

Create matrices...



### Constraints

Mandatory restrictions passed to the optimizer. We enforce two contraints:

* Only one coupon per user
* The total sum of the coupons should be less or equal than the total budget (10% of the CLTV)

In [8]:
# Constraints
print('Define constraints...\n')

print('\tRestriction 1 (Only one coupon per user): sum(x[i, :]) = 1...\n')

constraints = {i:
               plp.LpConstraint(
                   e=plp.lpSum(x[i, j]
                               for j in coupons_per_user[i]), 
                   sense=plp.LpConstraintEQ,
                   rhs=1*(len(coupons_per_user[i]) > 0),
                   name="constraint_{0}".format(i))
               for i in set_users}

print('\tRestriction 2: budget <= {}...\n'.format(b))

constraints[-1] = plp.LpConstraint(
    e=plp.lpSum(a[i, j] * x[i, j] for i in set_users for j in coupons_per_user[i]), 
    sense=plp.LpConstraintLE,
    rhs=b,
    name="constraint_b")

Define constraints...

	Restriction 1 (Only one coupon per user): sum(x[i, :]) = 1...

	Restriction 2: budget <= 75.1...



### Objective function

Function to be maximized (or minimized). In this problem, we are interested in maximizing the expected return after giving the optimal coupon, that is the user's CLTV minus the optimal coupon, given the probability of accepting this coupon:

$\max_{x: [0,1]} f(d,c,x) = d_ij * c_ij * x_ij$

In [9]:
# Objective function
print('Define objective function\n')

objective = plp.lpSum(x[i, j] * c[i, j] * d[i,j] for i in set_users for j in coupons_per_user[i])

# Putting all together - Creating optimization problem
print('Create optimization problem...\n')

opt_model = plp.LpProblem(name='Optimization Coupon Model')

for k in constraints.keys():
    opt_model.addConstraint(constraints[k])

opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)

Define objective function

Create optimization problem...



/home/wilomaku/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Solving

In [10]:
# Solving the optimization problem
opt_model.solve()

print('Problem {}!\n'.format(plp.constants.LpStatus[opt_model.status]))

print('\tPrepare coupon data\n')
df['coupon_opt'] = np.NaN
for var in opt_model.variables():

    if var.varValue == 1:
        i, j = int(var.name.split('_')[1]), int(var.name.split('_')[2])
        df.loc[i, 'coupon_opt'] = j
        
print('\tObjective function at optimal: {}'.format(opt_model.objective.value()))

Problem Optimal!

	Prepare coupon data

	Objective function at optimal: 152.86


In [11]:
# Adding optimization columns
df['prob_coupon_opt'] = df.apply(lambda x: x['prob_coupon_{}'.format(int(x['coupon_opt']))], axis=1)
df['roi_opt'] = df['prob_coupon_opt']*(df['cltv'] - df['coupon_opt'])
df[['cltv','coupon_bau','prob_coupon_bau','roi_bau','coupon_opt','prob_coupon_opt','roi_opt']]

,cltv,coupon_bau,prob_coupon_bau,roi_bau,coupon_opt,prob_coupon_opt,roi_opt
0,98.0,10,0.42,36.96,11.0,0.48,41.76
1,82.0,8,0.30,22.20,11.0,0.48,34.08
2,112.0,11,0.24,24.24,11.0,0.24,24.24
3,91.0,9,0.06,4.92,4.0,0.01,0.87
4,80.0,8,0.30,21.60,11.0,0.48,33.12
5,87.0,9,0.06,4.68,4.0,0.01,0.83
6,35.0,4,0.04,1.24,4.0,0.04,1.24
7,40.0,4,0.06,2.16,9.0,0.36,11.16
8,46.0,5,0.08,3.28,6.0,0.12,4.80
9,80.0,8,0.05,3.60,4.0,0.01,0.76


In [12]:
# Revenue expected for optimal coupons
ROI_OPT = df['roi_opt'].sum()
print('Expected ROI for optimizer: {}'.format(ROI_OPT))

Expected ROI for optimizer: 152.86


## Comparison BAU vs Optimization

Conclusion: The optimization of the coupons got 22% higher expected ROI over the classical BAU approach.

In [13]:
(ROI_OPT - ROI_BAU) / (ROI_BAU)

0.22405509288917377

## References

[1] The Perils of Social Coupon Campaigns. V. Kumar, B. Rajan. MIT Sloan. 2012. https://sloanreview.mit.edu/wp-content/uploads/2012/05/17a163f6ab.pdf

[2] PuLP. https://coin-or.github.io/pulp/main/index.html

[3] Linear programming and discrete optimization with Python using PuLP. T. Sarkar. Towards Data Science. 2019. https://towardsdatascience.com/linear-programming-and-discrete-optimization-with-python-using-pulp-449f3c5f6e99